# Классификация последовательностей (текстов) -- good practices

На прошлом семинаре мы рассмотрели пример генерации текстов. Однако мы не обсудили весь огромный функционал, который предоставляет пайторч для работы с текстами. В частности работали мы с посимвольной генерацией, а не пословной/потокенной. Исправляемся!

Сегодня речь зайдёт о задаче классификации текстовых последовательностей. Для этого будем пользоваться датасетом IMDB и библиотекой `torchtext`. В торчтексте реализовано огромное число методов для обработки текстов, ими мы и воспользуемся.

__АХТУНГ__. Торчтекст не рекомендуется использовать для обучения на больших данных (от миллиона примеров и больше) из-за маленькой скорости работы. В таких случаях рекомендуется имплементировать свои датасеты. В наших примерах таких объёмов данных не будет.

## Начнём с модели.

Да, не самая привычная последовательность действий, но давайте сначала сделаем кое-что, с чем мы уже знакомы -- имплементируем модель.

На вход она будет принимать батч последовательностей токенов, разумеется, разной длины в разных батчах, а на выходе выдавать батч вероятностей классов, прямо как в классификации картинок.

Каждый токен пройдёт вначале через эмбеддинг, а затем последовательность эмбеддингов пройдёт через LSTM. Самое последнее скрытое состояние будем считать векторным представлением для последовательности, поверх неё мы навесим линейный слой.

In [ ]:
import pandas as pd
import torch
from torch import nn


class TextClassifier(nn.Module):
    def __init__(
        self,
        num_embeddings=25002,
        embedding_size=300,
        hidden_size=200,
        num_classes=2,
        num_layers=1,
    ):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (last_hidden, last_c) = self.lstm(embedded)
        return self.linear(last_hidden[0]).squeeze()


model = TextClassifier()

In [ ]:
model(torch.tensor([[1, 2, 3, 4], [1, 2, 3, 4]]))

## Скачаем и проинициализируем датасет

Дальше перейдём к торчтексту. Он почти как знакомый нам `torchvision` имеет внутри себя коллекцию датасетов для разных задач NLP, в том числе и отзывы на IMDB. Инициализация датасета порой занимает время, не следует волноваться и судорожно перезапускать ядро.

In [ ]:
import torchtext
from torchtext.legacy import data
from torchtext.legacy.datasets import IMDB

In [ ]:
# воспользуемся старым добрым спейси
! pip install spacy
! python -m spacy download en_core_web_sm

В `torchtext` существует такая сущность, как `Field`. Это просто класс, в котором содержится описание колонок нашего датасета. В нашем случае всё довольно просто. Есть две колонки -- это текст и оценка, назовём их `text_field` и `label_field` соответственно. Токенизовать будем при помощи установленной только что библиотеки `spacy`, лейблы приведём к типу `float`. 

Напоминание, зачем нам batch_first: 

    If your input data is of shape (seq_len, batch_size, features) then you don’t need batch_first=True and your LSTM will give output of shape (seq_len, batch_size, hidden_size).

    If your input data is of shape (batch_size, seq_len, features) then you need batch_first=True and your LSTM will give output of shape (batch_size, seq_len, hidden_size).

In [ ]:
text_field = data.Field(
    tokenize="spacy",
    batch_first=True,
    include_lengths=False,
    tokenizer_language="en_core_web_sm",
)

label_field = data.LabelField(dtype=torch.float32, batch_first=True)

Создадим два сплита нашего датасета. Они задаются при помощи метода splits.

In [ ]:
data_train, data_test = IMDB.splits(text_field, label_field)

Создадим также словари, соответствующие нашему тексту. Выкинем все слова (токены), которые встречаются редко. Оставим только 25 тысяч самых частых слов.

Также согласуем номера токенов в словаре с эмбеддингами glove. При желании можно согласовать и с word2vec'ом, но имейте ввиду, что glove это их аналог. Об инициализации эмбеддингов мы обязательно поговорим чуть позже.

In [ ]:
vocab_size = 25000

# build_vocab -- создать словарь по данному полю в датасете
text_field.build_vocab(
    data_train,
    max_size=vocab_size,
    vectors="glove.6B.100d",
)

label_field.build_vocab(data_train)


for item in data_train:
    print(item.text)
    break

Давайте посмотрим на самые частые токены в словаре.

In [ ]:
text_field.vocab.freqs.most_common(10)

In [ ]:
# itos -- index to string
text_field.vocab.itos[:10]

Как мы видим, в поле текста у нас стоят токены, на которые поделил spacy поделил текст! Осталось только воспользоваться построенным словарём и сделать аналог даталоадера по данным!

In [ ]:
train_dataloader, test_dataloader = data.BucketIterator.splits(
    (data_train, data_test), batch_size=32, device="cuda:0"
)

In [ ]:
for item in train_dataloader:
    print(item.text.shape, item.label.shape)
    break

Будем обрезать их на 256 токенах в нашем трейн лупе.

## Train loop

Перейдём к самому интересному (нет) и построим трейн луп к нашей модели. Но скажем сразу, он мало чем будет отличаться от классификации картинок.

In [ ]:
def train_epoch(
    model,
    data_loader,
    optimizer,
    criterion,
    return_losses=False,
    device="cuda:0",
):
    model = model.to(device).train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    total_predictions = np.array([])  # .reshape((0, ))
    total_labels = np.array([])  # .reshape((0, ))
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for item in data_loader:
            texts = item.text
            labels = item.label

            # Move Batch to GPU
            texts = texts.to(device)
            labels = labels.to(device)
            predicted = model(texts)
            loss = criterion(predicted, labels)
            # Update weights
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            # Update descirption for tqdm
            accuracy = ((predicted > 0.5).int() == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(
                total_predictions, (predicted > 0.5).int().cpu().detach().numpy()
            )
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
            all_losses.append(loss.detach().item())
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    if return_losses:
        return metrics, all_losses
    else:
        return metrics


def validate(model, data_loader, criterion, device="cuda:0"):
    model = model.eval()
    total_loss = 0
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for texts, labels in data_loader:
            texts = texts.to(device)
            labels = labels.to(device)
            predicted = model(texts)
            loss = criterion(predicted, labels)
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(
                total_predictions, predicted.argmax(1).cpu().detach().numpy()
            )
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    return metrics

In [ ]:
import sys

import numpy as np
from tqdm.notebook import tqdm

device = "cuda:0"
model = TextClassifier()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

for i in range(10):
    train_epoch(
        model, train_dataloader, criterion=criterion, optimizer=optimizer, device=device
    )

Кажется, обучается так себе. Давайте это исправлять! Попробуем сделать модель двунаправленной, а также увеличим число слоёв. Должно быть существенно лучше!

In [ ]:
class TextClassifier(nn.Module):
    def __init__(
        self,
        num_embeddings=25002,
        embedding_size=300,
        hidden_size=200,
        num_classes=2,
        num_layers=1,
        pad_token=1,
    ):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(
            num_embeddings, embedding_size, padding_idx=pad_token
        )
        self.lstm = nn.LSTM(
            embedding_size,
            hidden_size,
            batch_first=True,
            num_layers=num_layers,
            bidirectional=True,
        )
        self.linear = nn.Linear(hidden_size * num_layers, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        _, (last_hidden, last_c) = self.lstm(embedded)
        hidden = torch.cat([last_hidden[-2], last_hidden[-1]], dim=1)
        return self.linear(hidden).squeeze()


device = "cuda:0"
pad_token = text_field.vocab.stoi["<pad>"]
model = TextClassifier(
    hidden_size=512, embedding_size=100, num_layers=2, pad_token=pad_token
)
model.embedding.weight.data = text_field.vocab.vectors
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.BCEWithLogitsLoss()

for i in range(10):
    train_epoch(
        model, train_dataloader, criterion=criterion, optimizer=optimizer, device=device
    )

model(torch.tensor([[1, 2, 3, 4], [1, 2, 3, 4]]))